### Spot Instances

**Execute a script**
https://peteris.rocks/blog/script-to-launch-amazon-ec2-spot-instances/

**Open a Notebook**
https://chrisalbon.com/aws/basics/run_project_jupyter_on_amazon_ec2/

*There are programmatic ways to retrieve both pricing and the launch wizard but these can take time so for a quick reference refer to the solutions below.*

**Prices** : https://aws.amazon.com/ec2/spot/pricing/
<br>**Images** : go to the launch-wizard to get the image IDs or use the boto3 client.describe_images() with Owners of Filters parameters to reduce wait time and find what you need.

**Storage**: storage options for spot instances include S3, EBS and EFS. Spot instances can be optimized for EBS. For a breakdown of storage options read [this](https://dzone.com/articles/confused-by-aws-storage-options-s3-ebs-amp-efs-explained) article. 

In [24]:
string = "b'l'b's\r\n'b'\x1b[0m\x1b[01;34manaconda3\x1b[0m        \x1b[01;34mexamples\x1b[0m               README  \x1b[01;34mtools\x1b[0m\r\n'b'\x1b[34;42mefs-mount-point\x1b[0m  Nvidia_Cloud_EULA.pdf  \x1b[01;34msrc\x1b[0m     \x1b[01;34mtutorials\x1b[0m\r\n'b'"

In [22]:
import re
ansi_escape = re.compile(r'\x1B[@-_][0-?]*[ -/]*[@-~]')

In [26]:
ansi_escape.sub('', string).encode('utf-8')

b"b'l'b's\r\n'b'anaconda3        examples               README  tools\r\n'b'efs-mount-point  Nvidia_Cloud_EULA.pdf  src     tutorials\r\n'b'"

In [7]:
import spot_connect as sc 

In [13]:
ssh_client = sc.connect_to_instance('52.36.226.213','KP-test',username='ec2-user')

Connecting...
Connected


In [14]:
session = ssh_client.get_transport().open_session()
session.get_pty()
session.invoke_shell()

In [15]:
interactive.interactive_shell(session)

Line-buffered terminal emulation. Press F6 or ^Z to send EOF.

       __|  __|_  )
       _|  (     /   Deep Learning AMI (Amazon Linux) Version 24.0
      ___|\___|___|

Please use one of the following commands to start the required environment with the framework of your choice:
for MXNet(+Keras2) with Python3 (CUDA 10.0 and Intel MKL-DNN) _____________________________________ source activate mxnet_p36
for MXNet(+Keras2) with Python2 (CUDA 10.0 and Intel MKL-DNN) _____________________________________ source activate mxnet_p27
for MXNet(+Amazon Elastic Inference) with Python3 _______________________________________ source activate amazonei_mxnet_p36
for MXNet(+Amazon Elastic Inference) with Python2 _______________________________________ source activate amazonei_mxnet_p27
for TensorFlow(+Keras2) with Python3 (CUDA 10.0 and Intel MKL-DNN) ___________________________ source activate tensorflow_p36
for TensorFlow(+Keras2) with Python2 (CUDA 10.0 and Intel MKL-DNN) ________________________

In [ ]:
def active_prompt(instance, user_name, port=22):
    client = connect_to_instance(instance['PublicIpAddress'],instance['KeyName'],username=user_name,port=port)
    print('Instance prompt open, using image OS. Type "exit" to end active prompt session')
    command='pwd'
    stdin, stdout, stderr = client.exec_command(command)                                      # Execute a command or .sh script (unix or linux console)
    try:
        currdir = ''
        for line in stdout:
            currdir+=line.rstrip()                                           # Show the output 
    except (KeyboardInterrupt, SystemExit):
        print(sys.stderr, 'Ctrl-C, stopping')

    while command!="exit":
        command = input(str(currdir)+' > ')
        stdin.write(command)
        stdin.flush()
        data = stdout.read.splitlines()
        for line in data:
            print(line.rstrip())
            
    client.close()                                                           # Close the connection 
    print('Exit code: 0')
    return True


In [55]:
import requests 
import time 
import boto3

region='us-west-2'
datasync_instance_name='datasync'
instance_filters={}
sg_filters={}
fs_creation_token=None
s3_bucket=None
use_first_bucket=True
use_first_role=True
datasync_wait=3

In [ ]:
client = boto3.client('ec2',region_name=region)                            # Connect to regional ec2 subnet 
instances = client.describe_instances()['Reservations']                    # Get the first instance with the keyname matching the datasync name 
instance = [instance['Instances'][0] for instance in instances if datasync_instance_name in instance['Instances'][0]['KeyName']][0]

owner_id = instance['NetworkInterfaces'][0]['OwnerId']                     # The owner of the instance is the owner of the entire account so we fix the owner id 

subnet_id = instance['SubnetId']                                           # Get the ID for the ec2 subnet 

sg = client.describe_security_groups(Filters=[sg_filters])['SecurityGroups'][0]
sg_id = sg['GroupId']                                                      # Get the GroupId for the security groups that match the filter parameters (first of all if no filters submitted)

In [ ]:
client = boto3.client('efs',region_name=region)                            # Connect to the elastic file system 
if fs_creation_token is None:
    filesystem = client.describe_file_systems()['FileSystems'][0]          # Retrieve the first file system 
else:                                                                      # or the first that matches the creation token submitted 
    filesystem = client.describe_file_systems(CreationToken=fs_creation_token)['FileSystems'][0]
fs_id = filesystem['FileSystemId']

mount_target = client.describe_mount_targets(FileSystemId=fs_id)['MountTargets'][0]
mount_ip = mount_target['IpAddress']

In [ ]:
client = boto3.resource('s3', region_name=region)                          # 
buckets = list(client.buckets.all())
if s3_bucket is None: 
    if use_first_bucket: 
        bucket_name = buckets[0].name
    else: 
        print('These are the S3 buckets in this region:')
        print(list(enumerate(buckets)))
        selection = input('Type the number of the bucket you want to use: ')
        bucket_name = buckets[int(selection)].name
else: 
    bucket_name = s3_bucket

In [ ]:
client = boto3.resource('iam', region_name=region)
roles = list(client.roles.all())
if use_first_role: 
    role_name = roles[0].name
else: 
    print("These are the roles in this region:")
    print(list(enumerate(roles)))
    selection = input('Type the number of the role you want to use: ')
    role_name = roles[int(selection)].name

In [ ]:
subnet_arn = 'arn:aws:ec2:'+region+':'+owner_id+':subnet/'+subnet_id
sg_arn = 'arn:aws:ec2:'+region+':'+owner_id+':security-group/'+sg_id
efs_arn = 'arn:aws:elasticfilesystem:'+region+':'+owner_id+':file-system/'+fs_id
bucket_arn = 'arn:aws:s3:::'+bucket_name
accessrole_arn = 'arn:aws:iam::'+owner_id+':role/'+role_name

In [ ]:
client = boto3.client('datasync',region_name=region)

online_agents = [agent for agent in client.list_agents()['Agents'] if agent['Status']=='ONLINE']
print('Retrieving online agent...')
if len(online_agents)==0:
    print('None found. Creating agent...')
    public_ip = instance['PublicIpAddress']
    r = requests.get("http://"+public_ip+"/?gatewayType=SYNC&activationRegion="+region+"&no_redirect")
    activation_key = r.text

    response = client.create_agent(
        ActivationKey=activation_key,
    )
    agent_arn = response['AgentArn']
    while len(online_agents)<1:
        print('.')
        online_agents = [agent for agent in client.list_agents()['Agents'] if agent['Status']=='ONLINE']    
        if len(online_agents)<1:
            time.sleep(datasync_wait)
        else: 
            print('Agent Online')
else: 
    agent_arn = online_agents[0]['AgentArn']
    print('Agent found')

In [ ]:
locations = client.list_locations()['Locations']
s3_locations = [location for location in locations if bucket_name in location['LocationUri']]
print('Retrieving S3 Location...')
if len(s3_locations)==0:
    response = client.create_location_s3(
        S3BucketArn=bucket_arn,
        S3Config={
            'BucketAccessRoleArn': accessrole_arn
        },
    )
    s3_location = response['LocationArn']
    while len(s3_locations)<1:
        print('.')
        locations = client.list_locations()['Locations']
        s3_locations = [location for location in locations if bucket_name in location['LocationUri']]
        if len(s3_locations)<1:
            time.sleep(datasync_wait)
        else: 
            print('S3 Location "%s" Retrieved' % str(bucket_name))
else: 
    s3_location = s3_locations[0]['LocationArn']
    print('S3 Location "%s" Retrieved' % str(bucket_name))

In [ ]:
print('Retrieving Elastic File System Location...')
locations = client.list_locations()['Locations']
fs_locations = [location for location in locations if mount_ip in location['LocationUri']]
if len(fs_locations)==0: 
    response = client.create_location_nfs(
        Subdirectory='/',
        ServerHostname=mount_ip,
        OnPremConfig={
            'AgentArns': [agent_arn]
        }
    )
    efs_location = response['LocationArn']
    while len(fs_locations)<1:
        print('.')
        locations = client.list_locations()['Locations']
        fs_locations = [location for location in locations if mount_ip in location['LocationUri']]
        if len(fs_locations)<1:
            time.sleep(datasync_wait)
        else: 
            print('EFS Location "%s" Online' % filesystem['CreationToken'])
else:
    efs_location = fs_locations[0]['LocationArn']
    print('EFS Location "%s" Online' % filesystem['CreationToken'])

In [11]:
client = boto3.client('datasync', region_name=region)
response = client.create_task(
    SourceLocationArn=s3_location,
    DestinationLocationArn=efs_location,
    Name='UploadToEfs',
)
task_arn = client.list_tasks()['Tasks'][0]['TaskArn']

In [13]:
response = client.start_task_execution(
    TaskArn=task_arn,
)
task_exec_arn = response['TaskExecutionArn']

In [15]:
client.describe_task_execution(TaskExecutionArn=task_exec_arn)

{'BytesTransferred': 0,
 'BytesWritten': 0,
 'EstimatedBytesToTransfer': 0,
 'EstimatedFilesToTransfer': 0,
 'Excludes': [],
 'FilesTransferred': 0,
 'Includes': [],
 'Options': {'Atime': 'BEST_EFFORT',
  'BytesPerSecond': -1,
  'Gid': 'INT_VALUE',
  'Mtime': 'PRESERVE',
  'PosixPermissions': 'PRESERVE',
  'PreserveDeletedFiles': 'PRESERVE',
  'PreserveDevices': 'NONE',
  'Uid': 'INT_VALUE',
  'VerifyMode': 'POINT_IN_TIME_CONSISTENT'},
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
   'content-length': '607',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Tue, 10 Sep 2019 16:12:26 GMT',
   'x-amzn-requestid': '2bc9fada-b81d-4a48-973a-a902dc8e1fda'},
  'HTTPStatusCode': 200,
  'RequestId': '2bc9fada-b81d-4a48-973a-a902dc8e1fda',
  'RetryAttempts': 0},
 'StartTime': datetime.datetime(2019, 9, 10, 11, 12, 10, 284000, tzinfo=tzlocal()),
 'Status': 'LAUNCHING',
 'TaskExecutionArn': 'arn:aws:datasync:us-west-2:006326790825:task/task-0055ebc20dce044ea/executio

# Scrumps